# Reading excel sheets into python as a matrix then multiply by an aggregation matrix to sum by category

#### Note this is only a proof of concept data is totally made up

In [239]:
import os
import pandas as pd
import numpy as np
import copy
from openpyxl import load_workbook
    
cwd = os.getcwd()
cwd = os.chdir('C:\Users\jojoseph\Desktop\Python Models\Python\IO')

print (os.getcwd())

C:\Users\jojoseph\Desktop\Python Models\Python\IO


In [240]:
#opening the social account matrix excel files from IMPLAN -(89 sectors summarized from study area data/study area explorer )
new_MRIO= 'new_MRIO.xlsx'# open file with social account matrices
reg_soc_acc_mat=pd.ExcelFile(new_MRIO)
print (reg_soc_acc_mat.sheet_names)

#region3=reg_soc_acc_mat.parse('region2')

[u'region0']


### Load aggregated sector names

In [241]:
sector_names = np.loadtxt('aggregated_sector_names.txt', usecols=1, delimiter="\'", dtype=str)[...,None]

sector_names

array([['Agriculture'],
       ['Mining'],
       ['Construction'],
       ['Durable Goods MFG'],
       ['Nondurable Goods MFG'],
       ['Trade, Transportaiton & Utilities'],
       ['Information'],
       ['Finance, Insurance & Real Estate'],
       ['Professional & Business Services'],
       ['Education & health Services'],
       ['Leisure & Hospitality'],
       ['Other Services'],
       ['Government'],
       ['Other Nonproducing Sectors']],
      dtype='|S33')

### Load the aggregation matrix

In [242]:
aggregation_matrix = np.loadtxt('aggregation_matrix.txt')

print aggregation_matrix

[[ 1.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


### For this exercise we will extract the first data column (sales) from each sheet and multiply it by the aggregation matrix

##### Note there are 534 rows in the excel sheets, the is because there are 6 regions (6x89=534) so we will just take the first region in our experiment

In [263]:
dict_of_df = {}
aggregated_dict = {}
for sheet_name in reg_soc_acc_mat.sheet_names:   
    df = reg_soc_acc_mat.parse(sheet_name)  # sheet name
    dict_of_df= df.iloc[:89,2:3]

dict_of_df

,Sales
0,2.282379e+09
1,1.335194e+09
2,6.537087e+09
3,1.319531e+09
4,6.073186e+09
5,1.013688e+10
6,2.659217e+10
7,1.386788e+10
8,8.560501e+09
9,7.480437e+07


# Load the Impact Conversion Factors

In [261]:
impact_conversion_matrix = np.loadtxt('impact_conversion_factors.txt')

print impact_conversion_matrix[0]
    

[  1.00000000e+00   1.00000000e+00   3.23420000e+00   5.38000000e-02
   5.56000000e-02   1.04400000e-01   1.00000000e-04]


In [264]:
Employment={}
Employee_Compensation={}
Proprietors_Income={}
Other_Property = {}
Indirect_Business_Tax = {}


for j in dict_of_df:
    sal = dict_of_df.Sales
    for k in range(0,89):
        #print sal[k]
        #print impact_conversion_matrix[k][3]pd.Series(A, index=data_frame.index)
        Employment[k] = (sal[k] * impact_conversion_matrix[k][2])/1000000
        Employee_Compensation[k] = (sal[k] * impact_conversion_matrix[k][3])
        Proprietors_Income[k] = (sal[k] * impact_conversion_matrix[k][4])
        Other_Property[k] = (sal[k] * impact_conversion_matrix[k][5])
        Indirect_Business_Tax[k] = (sal[k] * impact_conversion_matrix[k][6])

dict_of_df['Sales'] = pd.Series(sal)
dict_of_df['Employment'] = pd.Series(Employment)
dict_of_df['Employee_Compensation'] = pd.Series(Employee_Compensation)
dict_of_df['Proprietors_Income'] = pd.Series(Proprietors_Income)
dict_of_df['Other_Property'] = pd.Series(Other_Property)
dict_of_df['Indirect_Business_Tax'] = pd.Series(Indirect_Business_Tax)
dict_of_df

,Sales,Employment,Employee_Compensation,Proprietors_Income,Other_Property,Indirect_Business_Tax
0,2.282379e+09,7.381670e+03,1.227920e+08,1.269003e+08,2.382804e+08,2.282379e+05
1,1.335194e+09,2.128098e+04,2.777203e+07,1.903986e+08,1.994779e+08,0.000000e+00
2,6.537087e+09,5.163384e+04,2.471019e+08,1.289114e+09,1.161640e+09,0.000000e+00
3,1.319531e+09,1.204732e+03,5.792740e+07,6.597654e+05,9.197129e+07,2.771015e+06
4,6.073186e+09,1.665085e+04,5.556965e+08,9.109779e+06,3.024447e+08,1.336101e+07
5,1.013688e+10,1.734522e+04,8.656897e+08,1.013688e+07,7.987863e+08,3.649278e+07
6,2.659217e+10,4.506840e+04,2.108759e+09,2.393295e+07,1.247173e+09,6.116198e+07
7,1.386788e+10,1.194302e+04,8.292991e+08,4.160364e+06,4.215835e+08,6.333460e+09
8,8.560501e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,7.480437e+07,3.210080e+02,1.566404e+07,2.019718e+05,2.879968e+07,1.129546e+06


### now multiply the sales by the aggregation matrix.

In [317]:
for i in dict_of_df:
    #print dict_of_df[i]
    aggregated_dict[i] = np.matmul(aggregation_matrix, dict_of_df[i])

aggregated_dict

{'Employee_Compensation': array([  4.70634124e+08,   2.05444347e+09,   4.95760322e+08,
          6.38740714e+09,   6.87287062e+09,   7.21709743e+09,
          1.72831509e+09,   2.30285342e+09,   6.51335713e+09,
          5.66019114e+09,   2.22772447e+10,   2.64232754e+09,
          6.69128052e+08,   0.00000000e+00]),
 'Employment': array([   83605.55501485,    80293.41480503,    17492.02123726,
          117099.1575299 ,   124918.77162061,   184811.54783119,
           41831.0099541 ,    89338.58729187,   166805.98737651,
          150403.2238013 ,  1391799.97982191,   119154.36609336,
           11631.3887734 ,        0.        ]),
 'Indirect_Business_Tax': array([  2.28237895e+05,   1.48623321e+09,   1.58044398e+07,
          7.02516645e+08,   6.74992014e+09,   2.32955837e+09,
          1.76780288e+08,   1.40226323e+09,   2.53089137e+08,
          1.87670712e+08,   4.71046643e+09,   4.50356336e+08,
          0.00000000e+00,   0.00000000e+00]),
 'Other_Property': array([  1.59939871e+

### Build the output array.

#### This code can be streamlined

In [318]:
# Create a Pandas dataframe from some data.
outdf = pd.DataFrame(aggregated_dict)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
outdf.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Add a header format.
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#D7E4BC',
    'border': 1})

# Write the column headers with the defined format.
for col_num, value in enumerate(outdf.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
    
# Close the Pandas Excel writer and output the Excel file.
writer.save()